In [191]:
#This is a program to predit quality of wines (white+red wines). Here, I did use z score method to remove outliers
import pandas as pd
import numpy as np
white_wine=pd.read_csv('../../practice/winequality-white.csv', sep=';')
red_wine=pd.read_csv('../../practice/winequality-red.csv', sep=';')

In [192]:
# replace space with underline
white_wine.columns=white_wine.columns.str.replace(' ','_')
red_wine.columns=red_wine.columns.str.replace(' ','_')
#add a new column
red_wine['wine_type']='r'
white_wine['wine_type']='w'

In [193]:
#add a new column
red_wine['quality_label']=red_wine['quality'].apply(lambda x: 'low' if x <= 5 else 'medium' if x <= 7 else 'high')
red_wine['quality_label']=pd.Categorical(red_wine['quality_label'], categories=['low','high','medium'])
white_wine['quality_label']=white_wine['quality'].apply(lambda x: 'low' if x <= 5 else 'medium' if x <= 7 else 'high')
white_wine['quality_label']=pd.Categorical(white_wine['quality_label'], categories=['low','high','medium'])

In [194]:
#merge two files
total_wine=pd.concat([white_wine, red_wine])
total_wine= total_wine.set_index(np.arange(len(total_wine)))

In [195]:
# sI use z score method to remove outliers
def outliers_remov(df):
    for colu in total_wine.loc[:,'fixed_acidity':'quality']:
        mean_df=df[colu].mean()
        std_df=df[colu].std()
        df_out= df[(df[colu] <= 3*std_df+mean_df) & (df[colu] >= (mean_df-3*std_df))]
        df=df_out       
    return df_out


total_wine=outliers_remov(total_wine)
total_wine= total_wine.set_index(np.arange(len(total_wine)))



In [196]:
#encoding
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
#le0=OneHotEncoder(sparse=False)
#total_wine['wine_type']=le0.fit_transform(total_wine[['wine_type']])
#le0.inverse_transform(total_wine['wine_type'].unique()) ####?
total_wine['wine_type']=total_wine['wine_type'].apply(lambda x: 0 if x=='w' else 1)

le1=LabelEncoder()
total_wine['quality_label']=le1.fit_transform(total_wine['quality_label'])
print(le1.inverse_transform(total_wine['quality_label'].unique())) ###???

['medium' 'low' 'high']


In [197]:
total_wine['quality_label'].value_counts()

2    3620
1    2117
0     182
Name: quality_label, dtype: int64

In [198]:
# ML for quality of wine Decision tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score


model=DecisionTreeClassifier()
x=total_wine.drop(columns=['quality_label', 'quality'])
y=total_wine['quality_label']
scale=MinMaxScaler()
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.3, random_state=0)
x_train_n=scale.fit_transform(x_train)
x_train_n=pd.DataFrame(x_train_n, columns=x_train.columns)
x_test_n=scale.fit_transform(x_test)
x_test_n=pd.DataFrame(x_test_n, columns=x_test.columns)
model.fit(x_train_n,y_train)
predict=model.predict(x_test_n)
score=accuracy_score(y_test, predict)
print('accuracy is: ', score)
print("Confusion matrix:")
print(confusion_matrix(y_test, predict))
print(classification_report(y_test,predict))
cohen=cohen_kappa_score(y_test, predict)
print('cohen_kappa is: ', cohen)

accuracy is:  0.6503378378378378
Confusion matrix:
[[ 10   1  38]
 [  5 373 302]
 [ 54 221 772]]
              precision    recall  f1-score   support

           0       0.14      0.20      0.17        49
           1       0.63      0.55      0.59       680
           2       0.69      0.74      0.72      1047

    accuracy                           0.65      1776
   macro avg       0.49      0.50      0.49      1776
weighted avg       0.65      0.65      0.65      1776

cohen_kappa is:  0.3028166209524942


In [199]:
# ML for quality of wine  RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


model= RandomForestClassifier()
x=total_wine.drop(columns=['quality_label', 'quality'])
y=total_wine['quality_label']
scale=MinMaxScaler()
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.3, random_state=0)
x_train_n=scale.fit_transform(x_train)
x_train_n=pd.DataFrame(x_train_n, columns=x_train.columns)
x_test_n=scale.fit_transform(x_test)
x_test_n=pd.DataFrame(x_test_n, columns=x_test.columns)
model.fit(x_train_n,y_train)
predict=model.predict(x_test_n)
score=accuracy_score(y_test, predict)
print('accuracy is: ', score)
print("Confusion matrix:")
print(confusion_matrix(y_test, predict))
print(classification_report(y_test,predict))
cohen=cohen_kappa_score(y_test, predict)
print('cohen_kappa is: ', cohen)

accuracy is:  0.7618243243243243
Confusion matrix:
[[  9   0  40]
 [  0 417 263]
 [  2 118 927]]
              precision    recall  f1-score   support

           0       0.82      0.18      0.30        49
           1       0.78      0.61      0.69       680
           2       0.75      0.89      0.81      1047

    accuracy                           0.76      1776
   macro avg       0.78      0.56      0.60      1776
weighted avg       0.77      0.76      0.75      1776

cohen_kappa is:  0.4998438776400158


In [200]:
total_wine['wine_type'].value_counts()

0    4653
1    1266
Name: wine_type, dtype: int64

In [201]:
# ML for quality of wine  LinearDiscriminantAnalysis

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


model= LinearDiscriminantAnalysis()
x=total_wine.drop(columns=['quality_label', 'quality'])
y=total_wine['quality_label']
scale=MinMaxScaler()
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.3, random_state=0)
x_train_n=scale.fit_transform(x_train)
x_train_n=pd.DataFrame(x_train_n, columns=x_train.columns)
x_test_n=scale.fit_transform(x_test)
x_test_n=pd.DataFrame(x_test_n, columns=x_test.columns)
model.fit(x_train_n,y_train)
predict=model.predict(x_test_n)
score=accuracy_score(y_test, predict)
print('accuracy is: ', score)
print("Confusion matrix:")
print(confusion_matrix(y_test, predict))
print(classification_report(y_test,predict))
cohen=cohen_kappa_score(y_test, predict)
print('cohen_kappa is: ', cohen)

accuracy is:  0.7190315315315315
Confusion matrix:
[[  0   1  48]
 [  0 358 322]
 [  0 128 919]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        49
           1       0.74      0.53      0.61       680
           2       0.71      0.88      0.79      1047

    accuracy                           0.72      1776
   macro avg       0.48      0.47      0.47      1776
weighted avg       0.70      0.72      0.70      1776

cohen_kappa is:  0.3985311853338429


/Users/marziehbaes/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/marziehbaes/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/marziehbaes/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [202]:
# ML for quality of wine  adaboost

from sklearn.ensemble import AdaBoostClassifier

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix, classification_report



model= AdaBoostClassifier()
x=total_wine.drop(columns=['quality_label', 'quality'])
y=total_wine['quality_label']
scale=MinMaxScaler()
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.3, random_state=0)
x_train_n=scale.fit_transform(x_train)
x_train_n=pd.DataFrame(x_train_n, columns=x_train.columns)
x_test_n=scale.fit_transform(x_test)
x_test_n=pd.DataFrame(x_test_n, columns=x_test.columns)
model.fit(x_train_n,y_train)
predict=model.predict(x_test_n)
score=accuracy_score(y_test, predict)
print('accuracy is: ', score)
print("Confusion matrix:")
print(confusion_matrix(y_test, predict))
print(classification_report(y_test,predict))
cohen=cohen_kappa_score(y_test, predict)
print('cohen_kappa is: ', cohen)

accuracy is:  0.6891891891891891
Confusion matrix:
[[  2   0  47]
 [  1 354 325]
 [ 23 156 868]]
              precision    recall  f1-score   support

           0       0.08      0.04      0.05        49
           1       0.69      0.52      0.59       680
           2       0.70      0.83      0.76      1047

    accuracy                           0.69      1776
   macro avg       0.49      0.46      0.47      1776
weighted avg       0.68      0.69      0.68      1776

cohen_kappa is:  0.34982245914968735
